# Weverse

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.support.ui import WebDriverWait
import urllib.request
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import time
import os

def download_image(url, path, name):
    if not os.path.exists(path):
        os.makedirs(path)
    test = "01"
    # Convert back to a string and use zfill to pad with zeros
    fullname = str(name).zfill(len(test))+".jpg"
    urllib.request.urlretrieve(url,path+fullname)

# Edge WebDriver 的路徑，根據你的系統和 WebDriver 的位置進行修改
edge_driver_path = './msedgedriver.exe'
# 創建 Edge 瀏覽器實例
options = webdriver.EdgeOptions()
# 添加 headers
headers = {
    "User-Agent": "",
    "Authorization": "",
    "Origin": "",
    "Referer": ""
}
options.add_argument(f"user-agent={headers['User-Agent']}")
options.add_argument(f"Authorization={headers['Authorization']}") # 身份驗證
options.add_argument(f"Origin={headers['Origin']}")
options.add_argument(f"Referer={headers['Referer']}")
options.use_chromium = True # 指定使用 Chromium 作為底層的瀏覽器引擎
options.add_argument("--disable-popup-blocking") # 禁用彈出窗口阻擋功能
# options.add_argument("--headless")  # 無頭模式
options.add_argument("--mute-audio")  # 静音
options.add_experimental_option("prefs", {
    "download.default_directory": "E:\\偶像\\Kep1er\\影片\\", # 下載文件的默認保存目錄
    "download.prompt_for_download": False, # 禁用下載時的提示對話框
    "download.directory_upgrade": True, # 允許瀏覽器自動升級下載目錄
    "safebrowsing.enabled": True # 啟用安全瀏覽功能
})
# 啟動 Edge瀏覽器
driver = webdriver.Edge(service=EdgeService(executable_path=edge_driver_path), options=options)
# 設定相關 cookie
payload = {
    "we2_device_id": "",
    "we2_access_token": ""
}

# 訪問網頁並添加 cookie
url = 'https://weverse.io'
driver.get(url)
for name, value in payload.items():
    driver.add_cookie({'name': name, 'value': str(value), 'domain': '.weverse.io'})

# 訪問網頁
url = 'https://weverse.io/lesserafim/artist'
driver.get(url)
time.sleep(10)
class_name = 'ArtistPostListItemView_attachment_wrap__blcy5' #get href

# 下滑頁面載入後續資料
for i in range(4):
    elements = driver.find_elements(By.CLASS_NAME, class_name)
    links = []
    for element in elements:
        if element.get_attribute('href') not in links:
            links.append(element.get_attribute('href'))
    # 模擬下滑，滾動到頁面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 等待一段時間，確保頁面加載完成
    time.sleep(2)

# 根據儲存的貼文連接下載
for link in links:
    driver.get(link)
    time.sleep(10)
    # 發文者
    author = driver.find_element(By.CLASS_NAME, 'PostHeaderView_nickname__6Cb7X').text
    # 文章 ID
    postId = link.split("/")[-1]
    new_download_directory = "save_path"
    formatted_line = "author: {:<4}  postId: {:<9}  儲存路徑: {}".format(author, postId, new_download_directory)
    print(formatted_line)
    # 創建下載路徑，如果路徑不存在
    if not os.path.exists(new_download_directory):
        os.makedirs(new_download_directory)
    # 使用 JavaScript 修改下載目錄
    driver.execute_cdp_cmd('Page.setDownloadBehavior', {'behavior': 'allow', 'downloadPath': new_download_directory})
    # 等待一段時間確保路徑設置完成
    time.sleep(2)
    
    try:
        thumbnail = driver.find_element(By.CSS_SELECTOR, '.PostPreviewVideoThumbnailView_dimmed_letterbox__nDrwS')
    except Exception as e:
        # find image
        img = driver.find_element(By.CLASS_NAME, 'photo')
        actions = ActionChains(driver)
        actions.move_to_element(img).click().perform()
        time.sleep(1)
        # click next button and serach img src
        while True:
            try:
                next_button = driver.find_element(By.CLASS_NAME, 'flicking-arrow-next')
            except Exception as e:
                src = driver.find_elements(By.CLASS_NAME, 'ImageViewerView_image_content__aWEYs')
                count = 1
                for i in src:
                    download_image(i.get_attribute('src'),new_download_directory ,count)
                    count = count+1
                break
            # 使用 ActionChains 模擬鼠標移動到元素上
            actions = ActionChains(driver)
            actions.move_to_element(next_button).perform()
            time.sleep(1)
            # 點擊下載按鈕
            next_button.click()
            time.sleep(1)
        continue
    # find video 
    # 點擊載入影片
    # 使用ActionChains模擬鼠標動作
    actions = ActionChains(driver)
    actions.move_to_element(thumbnail).click().perform()
    time.sleep(5)
    download_button = driver.find_element(By.CLASS_NAME, 'ImageViewerView_download_button__9ipF7')
    # 使用 ActionChains 模擬鼠標移動到元素上
    actions = ActionChains(driver)
    actions.move_to_element(download_button).perform()
    time.sleep(1)

    # 點擊下載按鈕
    download_button.click()
    time.sleep(5)

# 關閉瀏覽器窗口
driver.quit()
print('end')